# Code Generation and Evaluation

In [1]:
from human_eval.data import write_jsonl, read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import random, time, datetime
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full,
                      reflection_prompt_basic, reflection_prompt, reflection_prompt_full, )
from helpers import clean_code_light, is_function            # write_jsonl

In [3]:
# select model, propmt, temperature, top_p
models = [
    ('ministral-3b-latest', 'mistral_3b'),
    ('ministral-8b-latest', 'mistral_8B'),
    ('open-mistral-nemo', 'nemo'),
    #('open-codestral-mamba', 'codestral_mamba'),
    #('open-mistral-7b', 'mistral_7b'),    
]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),            
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# reflection prompts & their names for log file name
reflection_prompts_and_names = [
    ( reflection_prompt_basic, 'reflection_prompt_basic' ),
    ( reflection_prompt, 'reflection_prompt' ),
    ( reflection_prompt_full, 'reflection_prompt_full' ), ]
# temperature and top_k values and labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'), ]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'), ]

model_idx            = 2
prompt_idx           = 3
relection_prompt_idx = 0
temperature_idx      = 0
top_p_idx            = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
my_reflection_prompt, my_reflection_prompt_label = reflection_prompts_and_names[ relection_prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:     {model_name}')
print(f'Model nickname: {model_nickname}')
print(f'Prompt name:    {my_prompt_label}')
print(f'Relfection prompt name: {my_reflection_prompt_label}')
print(f'Prompt:\n{my_prompt.strip().format("Task Description", "Test Cases")}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:     open-mistral-nemo
Model nickname: nemo
Prompt name:    complete_task_prompt_basic
Relfection prompt name: reflection_prompt_basic
Prompt:
Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

Temperature: 1.0. Temperature label: temperature1.0
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [4]:
models

[('ministral-3b-latest', 'mistral_3b'),
 ('ministral-8b-latest', 'mistral_8B'),
 ('open-mistral-nemo', 'nemo')]

In [5]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'logs_local/LBPP_{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'logs_local/logs/LBPP_{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
    logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
    logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-06-08 18:20:42,910  INFO  *****     JOB CONFIGURATION     *****
2025-06-08 18:20:42,910  INFO  MODEL: open-mistral-nemo
2025-06-08 18:20:42,911  INFO  MODEL NICKNAME: nemo
2025-06-08 18:20:42,911  INFO  Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-06-08 18:20:42,912  INFO  GENERATED CODE SAVED IN: logs_local/LBPP_nemo_complete_task_prompt_basic_temperature1.0_topP1.0_completions_20250608_182042_9099.jsonl
2025-06-08 18:20:42,912  INFO  THIS LOG FILE: logs_local/logs/LBPP_nemo_complete_task_prompt_basic_temperature1.0_topP1.0_log_20250608_182042_9099.log
2025-06-08 18:20:42,912  INFO  
PROMPT:

Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

2025-06-08 18:20:42,913  INFO  *****     END OF JOB CONFIGURATION     *****


In [6]:
# read dataset
file = 'data/LBPP_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 2:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: lbpp/0 <class 'str'>

task_id:
lbpp/0

test:
import numpy as np

ar = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
expected_output = np.array([[1, 2, 3, 2.0, 0.81649658], [4, 5, 6, 5.0, 0.81649658], [7, 8, 9, 8.0, 0.81649658]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1], [1], [2]])
expected_output = np.array([[1, 1, 0.0], [1, 1, 0.0], [2, 2, 0.0]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1, 2, 3, 4, 5]])
expected_output = np.array([[1, 2, 3, 4, 5, 3.0, 1.41421356]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)

prompt:
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You ma

In [7]:
def api_call(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model=model,
                messages=[{
                        "role": "user",
                        "content": prompt, 
                    }]
                )
    return chat_response.choices[0].message.content

def generate_response_with_backoff(model, prompt, max_retries=10, initial_delay=1, max_delay=16):
    '''
    Make API calls with exponential backoff on errors.
    Args:
        model (str): The model identifier.
        prompt (str): The prompt to send.
        max_retries (int): Maximum number of retries before giving up.
        initial_delay (int or float): Starting delay in seconds.
        max_delay (int or float): Maximum delay between retries.
    Returns:
        str: The chat response content OR f'Error generating a completion:{e}'
        if the max number of attemps has been reached
    '''
    retries = 0
    delay   = initial_delay
    while True:
        try:
            return api_call(model, prompt)
        except Exception as e:
            retries += 1
            if retries > max_retries:
                return f"Error generating a completion after {retries} retries:\n{e}"
            sleep_time = delay + random.uniform(0, 1)    # add jitter
            time.sleep(sleep_time)
            delay = min(delay * 2, max_delay)

In [8]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [9]:
input_prompt = "What is the capital of California?"
print(input_prompt)
print(generate_response_with_backoff(model_name, input_prompt))

What is the capital of California?
2025-06-08 18:20:44,188  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
The capital of California is Sacramento. It has been the state capital since 1854. Sacramento is known for its historic sites, such as the California State Capitol, Old Sacramento, and the Crocker Art Museum. It's also the sixth-largest city in California, with a population of around 500,000 people.


In [10]:
num_samples_per_task = 1
completions  = []

for task_id, task_body in tasks.items():
    logging.info( task_id )
    #if task_id == 'lbpp/5':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        
        # generate completion
        full_prompt = my_prompt.lstrip().format( task_body['prompt'], task_body['test'] )
        proposed_solution = generate_response_with_backoff( model_name, full_prompt )
        
        # improve solution
        new_full_prompt = my_reflection_prompt.lstrip().format(full_prompt, proposed_solution)
        improved_solution = generate_response_with_backoff( model_name, new_full_prompt )        
        
        # check if improved solution is a Python function. If not, use the previously proposed solution
        cleaned_improved_solution = clean_code_light(improved_solution).strip()
        if is_function(cleaned_improved_solution):
            final_solution = improved_solution
        else:
            final_solution = proposed_solution
            
        # save and log the results
        completions.append( {'task_id': task_id, 'completion': final_solution} )
        write_jsonl(results_file, completions)
        logging.info('NEW REFLECTION PROMPT:\n' + new_full_prompt + '\n')
        logging.info('IMPROVED COMPLETION:\n' + final_solution + '\n')
        logging.info(f"Time elapsed including backoff: {(time.time() - start_time):.4f} seconds\n")

2025-06-08 18:20:44,229  INFO  lbpp/0
2025-06-08 18:20:45,493  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:20:47,622  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:20:47,634  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Return an optimized version of the PROPOSED SOLUTION.
3. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You may assume that the input array has at least one row and one column..

Your output code must satisfy the

2025-06-08 18:20:55,843  INFO  Time elapsed including backoff: 8.2062 seconds

2025-06-08 18:20:55,844  INFO  lbpp/2
2025-06-08 18:20:59,825  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:21:03,276  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:21:03,291  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Return an optimized version of the PROPOSED SOLUTION.
3. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
You are given a target word and an array of candidate words. Write a Python program to pick a subset of words from the candidate words such that 1) You can form the target word from the subset of candidate words by re-arranging some or all the letters of the candidate words. 2) The number of unused letters from the subset of candidate words is mi

2025-06-08 18:21:09,775  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and satisfies the given requirements and tests. There is no need for optimization in this case. Here's the proposed solution again for your reference:

```python
from collections import namedtuple
from typing import List

Products = namedtuple("Products", ["name", "category", "price", "quantity"])

def apply_discount(order_list: List[Products], category: str, discount: float) -> float:
    total_price = 0
    for product in order_list:
        if product.category == category:
            total_price += product.price * product.quantity * (1 - discount)
        else:
            total_price += product.price * product.quantity
    return total_price

# Test cases
product1 = Products(name="Laptop", category="Electronics", price=1200, quantity=10)
product2 = Products(name="Smartphone", category="Electronics", price=800, quantity=25)
product3 = Products(name="Backpack", category="Fashion", price=50, 

2025-06-08 18:21:16,830  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal in terms of time and space complexity. It has a time complexity of O(n) and a space complexity of O(n), where n is the total number of students. The solution uses prefix sum and binary search, which are efficient algorithms for this problem.

Here's the proposed solution with some minor formatting improvements:

```python
def arrange_grades(students):
    prefix_sum = [0] * len(students)
    for i in range(len(students)):
        prefix_sum[i] = sum(students[i])

    for i in range(1, len(prefix_sum)):
        if prefix_sum[i] <= prefix_sum[i-1]:
            return False

    return True
```

Since the proposed solution is already optimal, I will return it as is.

2025-06-08 18:21:16,831  INFO  Time elapsed including backoff: 3.7806 seconds

2025-06-08 18:21:16,832  INFO  lbpp/6
2025-06-08 18:21:18,260  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-0

2025-06-08 18:21:26,264  INFO  Time elapsed including backoff: 6.2409 seconds

2025-06-08 18:21:26,265  INFO  lbpp/8
2025-06-08 18:21:28,563  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:21:48,563  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:21:48,578  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Return an optimized version of the PROPOSED SOLUTION.
3. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a python function "def average_grades(grades: pd.DataFrame) -> List[Tuple[float, str]]" that returns the average student's grade by subject. Do not include the Arts class in the calculations and handle negative or null grades, changing them to zero. If the average grade is above 80, include a "*" in before the subject's name in the output a

...

2025-06-08 18:38:51,765  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and satisfies all the requirements. It uses the pandas library to read the CSV file and perform data validation, which is efficient and convenient for this task. The function also includes error handling to detect and raise appropriate errors when there's a problem with the imported data.

Here's the proposed solution again for reference:

```python
import csv
import pandas as pd

def total_sales(file_path: str) -> float:
    try:
        # Try to read the CSV file using pandas
        df = pd.read_csv(file_path)
    except pd.errors.ParserError as e:
        # If there's a problem with the CSV file, raise a ValueError
        raise ValueError(f"Error reading CSV file: {e}")

    # Check if the required columns are present
    required_columns = ["order_id", "product_id", "unit_price", "quantity"]
    if not all(col in df.columns for col in required_columns):
        raise ValueError("CSV file 

2025-06-08 18:39:04,973  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correctly implements the required functionality. It uses the cross product of vectors to determine if the origin is inside the triangle formed by the given points. The cross product of two vectors is zero if and only if the vectors are parallel, which is the case if the origin lies on the line segment connecting the two points.

Here's the proposed solution with some minor formatting improvements and added type hints:

```python
from math import sqrt
from typing import Tuple

def does_triangle_contain_origin(p1: Tuple[float, float], p2: Tuple[float, float], p3: Tuple[float, float]) -> bool:
    def cross_product(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
        return p1[0] * p2[1] - p1[1] * p2[0]

    def dot_product(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
        return p1[0] * p2[0] + p1[1] * p2[1]

    def vector(p1: Tuple[float, float], p2: Tuple[f

2025-06-08 18:39:13,064  INFO  Time elapsed including backoff: 8.0883 seconds

2025-06-08 18:39:13,064  INFO  lbpp/158
2025-06-08 18:39:14,791  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:39:17,146  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:39:17,159  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Return an optimized version of the PROPOSED SOLUTION.
3. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a python function "production_cost(raw_material_cost: Dict[str,float], raw_material_usage: Dict[str,float], daily_production: List[int]) -> float" function that takes as input two dictionaries: one containing
the cost of each raw material and the other containing the number of each
raw material that are used to produce one unit. Additiona

2025-06-08 18:39:26,992  INFO  Time elapsed including backoff: 9.8299 seconds

2025-06-08 18:39:26,993  INFO  lbpp/160
2025-06-08 18:39:29,894  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:39:30,968  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-08 18:39:30,985  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Return an optimized version of the PROPOSED SOLUTION.
3. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a Python function named validate_spiral that checks whether a given 2D array with dimensions N x M (where N and M are the number of rows and columns, respectively) is a valid spiral.

- The spiral should start in the top-left corner and spiral clockwise towards the center.
- Each element in the spiral must be in ascending order, increment

2025-06-08 18:39:37,595  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal. It has a time complexity of O(n*m*k), where n is the number of words, m is the number of ranges, and k is the length of the words. This is because for each word, we iterate over each character and check if it is in the corresponding range, which takes O(k) time. Since we do this for all words, the overall time complexity is O(n*k). The space complexity is O(n), as we store the valid words in a set.

Here is the proposed solution again for reference:

```python
def word_range(words, ranges):
    valid_words = set()
    for word in words:
        if all(word[i] in ranges[i] for i in range(len(word))):
            valid_words.add(word)
    return valid_words
```

And here are the test cases again:

```python
assert word_range(["apple", "banan", "cherr", "dates", "elder", "figss", "grape"], ["a-c", "h-p", "b-q","j-s","c-t"]) == {"apple", "cherr"}
assert word_range(["apple", "banan", "cherr", "date

In [11]:
unknown_variable

NameError: name 'unknown_variable' is not defined

# Appendix

In [ ]:
import requests

# Set your Mistral API key
API_KEY = api_key

url = "https://api.mistral.ai/v1/models"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    models = response.json()
    #print("Available models:", models)
else:
    print("Failed to retrieve models:", response.status_code, response.text)

In [ ]:
for k, v in models.items():
    print(k)
    print(v)
    print("="*100)

In [ ]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [ ]:
data[0]['completion']

In [ ]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [ ]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)